In [32]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, f1_score
)
import numpy as np
import pandas as pd
import sys, os

# ====== YOUR PREPROCESSOR ======
sys.path.append(os.path.abspath("../.."))
from src.preprocessing import preprocess_text_fin


In [33]:
# ====== DATA ======
DATA_PATH = "../../data/data.csv"

# 1) Cấu hình chung (áp dụng cho mọi mô hình)

**Lý do thiết kế TF-IDF & CV:**

* **`ngram_range=(1,2)`**: câu ngắn → bigram bắt cụm giàu tín hiệu (“rose sharply”, “beat estimates”).
* **`min_df∈{3,5,10}`**: lọc từ quá hiếm do tập chỉ \~5.8k câu.
* **`max_df∈{0.9,0.95}`**: loại từ quá phổ biến (gần stopword).
* **`max_features∈{5k,10k,15k}`**: cân bằng bias/variance với cỡ dữ liệu \~5.8k.
* **`sublinear_tf=True`**: giảm “burstiness” của từ xuất hiện nhiều.
* **`norm='l2'`**: phù hợp linear models trên dữ liệu sparse.

In [34]:
# Không chỉnh tokenizer trong grid để tránh lỗi set_params
BASE_TFIDF = TfidfVectorizer(
    tokenizer=word_tokenize,   # gắn với tiền xử lý text tiếng Anh
    token_pattern=None,
    use_idf=True
)

tfidf_space = {
    "tfidf__ngram_range": [(1,1), (1,2)],       #ngram_range=(1,2): câu ngắn → bigram bắt cụm giàu tín hiệu (“rose sharply”, “beat estimates”).
    "tfidf__min_df": [3, 5, 10],                #min_df∈{3,5,10}: lọc từ quá hiếm do tập chỉ ~5.8k câu.
    "tfidf__max_df": [0.90, 0.95],              #max_df∈{0.9,0.95}: loại từ quá phổ biến (gần stopword).
    "tfidf__max_features": [5000, 10000, 15000],#max_features∈{5k,10k,15k}: cân bằng bias/variance với cỡ dữ liệu ~5.8k.
    "tfidf__sublinear_tf": [True],              #sublinear_tf=True: giảm “burstiness” của từ xuất hiện nhiều.
    "tfidf__norm": ["l2"],                      #norm='l2': phù hợp linear models trên dữ liệu sparse.
}

# 2) Logistic Regression + TF-IDF (baseline mạnh)

**Lý do chọn & tham số:**

* LR rất hợp với vector TF-IDF high-dimensional; nhanh, ổn định.
* **`C`** điều khiển regularization: dữ liệu 5.8k → cần quét từ chặt đến lỏng.
* **`solver='liblinear'` + `multi_class='ovr'`**: ổn với sparse và 3 lớp.
* **`class_weight='balanced'`**: bù lệch lớp

In [35]:
from sklearn.linear_model import LogisticRegression

pipe_lr = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", OneVsRestClassifier(
        LogisticRegression(
            solver="liblinear",
            penalty="l2",
            class_weight="balanced",
            max_iter=4000,
        ),
        n_jobs=-1
    ))
])


param_grid_lr = {
    **tfidf_space,
    "clf__estimator__C": [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
}

# 3) Linear SVM (LinearSVC) + TF-IDF (mạnh với dữ liệu sparse)

**Lý do & tham số:**

* Linear SVM thường cho biên tốt trên TF-IDF.
* **`C`** quét rộng để tìm margin tối ưu.
* **`loss='squared_hinge'`** là chuẩn cho LinearSVC.
* **`class_weight='balanced'`** xử lý lệch.


In [36]:
from sklearn.svm import LinearSVC

pipe_svm = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", LinearSVC(
        loss="squared_hinge",
        class_weight="balanced",
        max_iter=4000
    ))
])

param_grid_svm = {
    **tfidf_space,
    "clf__C": [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
}

# 4) Multinomial Naive Bayes + TF-IDF (baseline cực nhanh)

**Lý do & tham số:**

* MNB phù hợp xác suất từ; nhanh, nhẹ để làm baseline.
* **`alpha`** (Laplace/Lidstone) rất quan trọng vì từ hiếm xuất hiện nhiều trong text ngắn → quét log-scale.
* **`fit_prior`** bật/tắt để xem tác động của phân bố lớp lệch.


In [37]:
from sklearn.naive_bayes import MultinomialNB

pipe_mnb = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", MultinomialNB())
])

param_grid_mnb = {
    **tfidf_space,
    "clf__alpha": [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    "clf__fit_prior": [True, False],
}

# 5) Random Forest + TF-IDF (khi có thêm feature phi-ngôn-ngữ)

**Lý do & tham số:**

* RF không lý tưởng nếu **chỉ** TF-IDF (quá high-dim). Tuy nhiên, nếu sau này anh **gộp thêm feature thống kê** (độ dài câu, số tiền, có/không dấu %) thì RF phát huy.
* Dù vậy, để hoàn chỉnh benchmark, ta vẫn cho grid gọn, có **`class_weight='balanced_subsample'`** để bù lệch.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
pipe_rf = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("svd", TruncatedSVD(random_state=42)),
    ("clf", RandomForestClassifier(
        n_jobs=-1,
        class_weight="balanced_subsample",
        random_state=42
    ))
])

param_grid_rf = {
    **tfidf_space,
    # SVD
    "svd__n_components": [200, 300],

    # RF (gọn mà hiệu quả)
    "clf__n_estimators": [300],        # cố định
}

# 5) GridSearchCV

In [38]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def run_gs(pipe, grid, X_train, y_train):
    gs = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        scoring="f1_macro",
        cv=cv,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)
    return gs

# 6) Preprocessing 

In [39]:
def preprocess_csv(csv_file: str) -> pd.DataFrame:
    df = pd.read_csv(csv_file)
    df["labels"] = LabelEncoder().fit_transform(df["Sentiment"])
    df.drop_duplicates(subset=["Sentence"], keep="first", inplace=True)
    df["Sentence"] = df["Sentence"].astype(str).apply(preprocess_text_fin)
    return df

In [40]:
df = preprocess_csv(DATA_PATH)
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df["Sentence"]),
    np.array(df["labels"]), 
    test_size=0.25, 
    random_state=42
    )

In [51]:
gs_lr = run_gs(pipe_lr, param_grid_lr, X_train, y_train)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


In [52]:
gs_svm = run_gs(pipe_svm, param_grid_svm, X_train, y_train)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


In [ ]:
gs_mnb = run_gs(pipe_mnb, param_grid_mnb, X_train, y_train)

Fitting 5 folds for each of 504 candidates, totalling 2520 fits


In [68]:
gs_rf = run_gs(pipe_rf, param_grid_rf, X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Users\hung0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  warnings.warn(


In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_auc_score
)
from sklearn.utils.multiclass import unique_labels

# 1) Tóm tắt kết quả Cross-Validation (best) của nhiều GridSearch
def summarize_cv_results(gridsearch_dict):
    rows = []
    for name, gs in gridsearch_dict.items():
        rows.append({
            "model": name,
            "cv_best_f1_macro": gs.best_score_,
            "best_params": gs.best_params_
        })
    df = pd.DataFrame(rows).sort_values("cv_best_f1_macro", ascending=False)
    print("\n=== CV Best Summary (sorted by f1_macro) ===")
    print(df.to_string(index=False))
    return df

# 2) Đánh giá chi tiết trên tập test cho 1 GridSearch đã fit
def evaluate_on_test(gs, X_test, y_test, model_name="Model"):
    best_est = gs.best_estimator_
    y_pred   = best_est.predict(X_test)

    # Thử lấy score cho ROC-AUC macro (ưu tiên predict_proba, fallback decision_function)
    y_score = None
    has_proba = hasattr(best_est, "predict_proba")
    has_decision = hasattr(best_est, "decision_function")
    if has_proba:
        try:
            y_score = best_est.predict_proba(X_test)
        except Exception:
            y_score = None
    if (y_score is None) and has_decision:
        try:
            y_score = best_est.decision_function(X_test)
        except Exception:
            y_score = None

    # Metrics cơ bản
    acc   = accuracy_score(y_test, y_pred)
    f1m   = f1_score(y_test, y_pred, average="macro")
    precm = precision_score(y_test, y_pred, average="macro", zero_division=0)
    recm  = recall_score(y_test, y_pred, average="macro", zero_division=0)

    # ROC-AUC macro (nếu có y_score nhiều lớp)
    rocauc = None
    if y_score is not None:
        try:
            rocauc = roc_auc_score(y_test, y_score, multi_class="ovo", average="macro")
        except Exception:
            rocauc = None

    # In tổng quan
    print(f"\n================= {model_name} =================")
    print("Best params:", gs.best_params_)
    print(f"CV best f1_macro: {gs.best_score_:.4f}")
    print(f"Test Accuracy   : {acc:.4f}")
    print(f"Test F1-macro   : {f1m:.4f}")
    print(f"Test Precision-m: {precm:.4f}")
    print(f"Test Recall-m   : {recm:.4f}")
    if rocauc is not None:
        print(f"Test ROC-AUC-m  : {rocauc:.4f}")

    # Classification report
    print("\nClassification report:")
    print(classification_report(y_test, y_pred, digits=4, zero_division=0))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    labels = unique_labels(y_test, y_pred)
    print("Confusion matrix (rows=true, cols=pred):")
    print(pd.DataFrame(cm, index=[f"true_{l}" for l in labels],
                          columns=[f"pred_{l}" for l in labels]).to_string())
    return {
        "model": model_name,
        "cv_best_f1_macro": gs.best_score_,
        "test_accuracy": acc,
        "test_f1_macro": f1m,
        "test_precision_macro": precm,
        "test_recall_macro": recm,
        "test_roc_auc_macro": rocauc,
        "best_params": gs.best_params_
    }

# 3) Chạy đánh giá cho tất cả mô hình + bảng tổng hợp test
def evaluate_all(gs_dict, X_test, y_test):
    _ = summarize_cv_results(gs_dict)
    test_rows = []
    for name, gs in gs_dict.items():
        res = evaluate_on_test(gs, X_test, y_test, model_name=name)
        test_rows.append(res)
    test_df = pd.DataFrame(test_rows).sort_values("test_f1_macro", ascending=False)
    print("\n=== Test Summary (sorted by f1_macro) ===")
    print(test_df[["model","cv_best_f1_macro","test_f1_macro","test_accuracy","test_precision_macro","test_recall_macro","test_roc_auc_macro"]].to_string(index=False))
    return test_df


In [69]:
gs_dict = {
    "Logistic Regression": gs_lr,
    "LinearSVM": gs_svm,
    "MultinomialNB": gs_mnb,
    "RandomForest": gs_rf,
    # nếu có GradientBoosting: "GradBoost": gs_gb,
}
test_summary = evaluate_all(gs_dict, X_test, y_test)


=== CV Best Summary (sorted by f1_macro) ===
              model  cv_best_f1_macro                                                                                                                                                                                                                            best_params
Logistic Regression          0.651727                                                      {'clf__estimator__C': 1.0, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
          LinearSVM          0.646353                                                                 {'clf__C': 0.5, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
      MultinomialNB          0.626157                                    {'clf__alpha': 0.5, 'clf__fit_prior': False, 'tfidf__max_df': 0.9, 'tfidf_